In [25]:
# FIRST CELL
# VERIFIED AS WORKING

# import dependencies
import pandas as pd

# read raw data
path = "purchase_data.csv" 
raw_data_df = pd.read_csv(path, encoding="UTF-8")
raw_data_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [26]:
# SECOND CELL
# FIND PLAYER COUNT
# VERIFIED AS WORKING
# code sourced from https://stackoverflow.com/questions/15943769/how-do-i-get-the-row-count-of-a-pandas-dataframe

# Find number of players - only counting each player once.
total_rows = raw_data_df["SN"].nunique()
f"Total Number of Players: {total_rows}"

'Total Number of Players: 576'

In [27]:
# THIRD CELL
# FIND PURCHASING ANALYSIS TOTALS
# VERIFIED AS WORKING AND FORMATTED CORRECTLY

# find number of unique items - counting each unique item only 1 time
unique_items_list = raw_data_df["Item ID"].unique()
unique_items_count = len(unique_items_list)
#print(f"Total Number of Unique Items: {unique_items_count}")

# Summary Table
summary_table = pd.DataFrame(
    {"Number of Unique Items": [unique_items_count],
     "Average Price": ['${0:.3}'.format(raw_data_df["Price"].mean())],
     "Number of Purchases": [raw_data_df["Price"].count()],
     "Total Revenue": [(raw_data_df["Price"].sum())]
    })
# print(summary_table)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map('${:.2f}'.format)
summary_table

# DEBUG total revenue:  .map('${:.2f}'.format)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [37]:
# FOURTH CELL
# FIND GENDER DEMOGRAPHICS
# VERIFIED AS WORKING AND FORMATTED CORRECTLY - NEED TO RENAME COLUMN SN as Total Count

# Count how many time Male, Female, or Other - counting only 1 time for each player
gender_demo_df = pd.DataFrame(raw_data_df.groupby(["Gender"])["SN"].nunique())

# Rename Column SN as Total Count - Not working - why???
gender_demo_df = gender_demo_df.rename(columns ={"SN": "Total Count"})

# Find the percentage of players
gender_demo_df["Percentage of Players"] = (gender_demo_df["Total Count"] / total_rows).map('{:.2%}'.format)

#QUESTION: '{0:.3%}'.format(gender_demo_df["SN"] / total_rows) does not work, but .map('{:.2%}'.format) does - why

# Sort dataframe by percentage of players
gender_demo_df = gender_demo_df.sort_values(["Percentage of Players"], ascending=False)
gender_demo_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [43]:
# FIFTH CELL
# PURCHASING ANALYSIS (GENDER):  PURCHASING ANALYSIS STATS BROKEN DOWN BY GENDER
# VERIFIED AS WORKING - VERIFIED FORMATTING

# Establish first column (Purchase Count) in the data frame
purchase_count_by_gender = pd.DataFrame(raw_data_df["Gender"].value_counts())
purhcase_count_by_gender = purchase_count_by_gender.rename(columns={"Gender":"Purchase Count"})
purchase_count_by_gender

# Establish 2nd column (Average Purchase Price) in the data frame
gender_purchases= (raw_data_df[["Gender", "Price"]])
avg_grouped_price_by_gender = gender_purchases.groupby('Gender').mean()
purchase_count_by_gender["Average Purchase Price"] = avg_grouped_price_by_gender
purchase_count_by_gender

# Establish 3rd column (Total Purchase Value) in the data frame
sum_grouped_price_by_gender = gender_purchases.groupby('Gender').sum()
purchase_count_by_gender["Total Purchase Value"] = sum_grouped_price_by_gender
purchase_count_by_gender

# Establish 4th column (Avg Total Purchase per Person) in the data frame
total_grouped_price_per_person = pd.DataFrame(raw_data_df.groupby(["SN", "Gender"]).sum())
avg_price_per_person_grouped_by_gender = pd.DataFrame(total_grouped_price_per_person.groupby(["Gender"])["Price"].mean())
purchase_count_by_gender["Avg Total Purchase per Person"] = avg_price_per_person_grouped_by_gender

# Reformat columns to 2 decimals with $
purchase_count_by_gender["Average Purchase Price"] = purchase_count_by_gender["Average Purchase Price"].map('${:.2f}'.format)
purchase_count_by_gender["Total Purchase Value"] = purchase_count_by_gender["Total Purchase Value"].map('${:.2f}'.format)
purchase_count_by_gender["Avg Total Purchase per Person"] = purchase_count_by_gender["Avg Total Purchase per Person"].map('${:.2f}'.format)
purchase_count_by_gender

#resort by index
purchase_count_by_gender = purchase_count_by_gender.sort_index()
purchase_count_by_gender

,Gender,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [50]:
# SIXTH CELL
# FIND AGE DEMOGRAPHIC
# VERIFIED AS WORKING - VERIFIED FORMATTING

# Another way to remove duplicates and keep columns SN, Age, and Price
age_demo_df = raw_data_df.filter(["SN","Age", "Price"], axis=1)
#no_dupe_age_demo_df = age_demo_df.drop_duplicates()

# Establish bins for ages and names for the bins. NOTE: used full upper limit excluding right value.
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]

# Create the names for the bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Establish bins in dataframe under field "Total Count".  NOTE: used full upper limit excluding right value.
age_demo_df["Age Bins"] = pd.cut(age_demo_df["Age"], bins, labels=group_names, include_lowest=True, right=False)

# create a new df and count by Age Bin - for unique values
grouped_age_demo_df = age_demo_df.groupby("Age Bins").nunique()

# Rename SN as Total Count
grouped_age_demo_df = grouped_age_demo_df.rename(columns ={"SN": "Total Count"})

# Remove unneeded columns
del grouped_age_demo_df["Age Bins"]
del grouped_age_demo_df["Age"]
del grouped_age_demo_df["Price"]

# Find total number of users
total_number_of_users = grouped_age_demo_df["Total Count"].sum()

# Find percentage of players
grouped_age_demo_df["Percentage of Players"] = (grouped_age_demo_df["Total Count"] / total_number_of_users)

# Reformat columns
grouped_age_demo_df["Percentage of Players"] = grouped_age_demo_df["Percentage of Players"].map('{:.2%}'.format)
grouped_age_demo_df

,Total Count,Percentage of Players
Age Bins,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [12]:
# SEVENTH CELL
# FIND PURCHASING STATS BY AGE BIN
# VERIFIED WORKING - VERIFIED FORMATTING.  -- Clean up code
# NOTE: ONLY LAST COLUMN DOES NOT MATCH SOLUTION, BUT LOGIC OF THIS CODE HAS BEEN VERIFIED

# Start with all rows of purchases with just the columns needed:  removed duplicates and keep columns SN, Age, and Price
age_demo_df

# Establish bins for ages and names for the bins
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]

# Create the names for the bins
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Establish bins in dataframe under field "Total Count"
purchase_demo_df = age_demo_df
purchase_demo_df["Purchase Counts"] = pd.cut(age_demo_df["Age"], bins, labels=group_names, include_lowest=True, right=False)

# Count each bin
purchase_stats_df = pd.DataFrame(purchase_demo_df["Purchase Counts"].value_counts())

# Find Average Purchase Price for each bin
average_purch_price_age = purchase_demo_df

# Resort dataframe
purchase_stats_df = purchase_stats_df.sort_index()
purchase_stats_df

# Create dataframe with Average Purchase Price for each Age Bin
average_purchase_df = purchase_demo_df.groupby("Purchase Counts").mean()
average_purchase_df

# Create dataframe with Total Purchase Price for each Age Bin
# DEBUG - CHANGED "Purchase Counts to "Price"??? = .  Should sum on Price
# DEBUG - len(total_purchase_df) = 576 should be 8
total_purchase_df = purchase_demo_df.groupby("Purchase Counts").sum()
total_purchase_df

# Create dataframe with Average Purchase Total per Person for each Age Bin
# Take purchase_demo_df and group by(SN) and sum
# Create age bins again
# Then group by (age bin) and perform mean
average_purch_total_df = purchase_demo_df.groupby("SN").sum()

# Establish bins for ages and names for the bins - for average_purch_total_df
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Establish bins in dataframe under field "Total Count"
average_purch_total_df["Age Bins"] = pd.cut(average_purch_total_df["Age"], bins, labels=group_names, include_lowest=True, right=False)
average_purch_total_df

# Find number of people in each age bin
# average_purch_total_per_person_df
age_counts_df = average_purch_total_df.groupby("Age Bins").count()
age_counts_df

# Group By SN (username/ screen name) and then sum.  The Price column should be the total purchase for each user.
avg_purch_per_person_df = pd.DataFrame(raw_data_df.groupby("SN").sum())
avg_purch_per_person_df

# Create Bins for the Age of users
avg_purch_per_person_df["Age Bins"] = pd.cut(avg_purch_per_person_df["Age"], bins, labels=group_names, include_lowest=True, right=False)
avg_purch_per_person_df

# Group avg_purch_per_person_df by Age Bins. The Price column should be the average of the (total purchase per person) in each age group.
avg_purch_per_person_df = avg_purch_per_person_df.groupby("Age Bins").mean()
avg_purch_per_person_df

# Delete unneeded columns
del avg_purch_per_person_df["Purchase ID"]
del avg_purch_per_person_df["Age"]
del avg_purch_per_person_df["Item ID"]
avg_purch_per_person_df

# Combine stats into 1 dataframe
# Fill out binned dataframe
purchase_stats_df["Average Purchase Price"] = average_purchase_df["Price"]
purchase_stats_df["Total_Purchase_value"] = total_purchase_df["Price"]
purchase_stats_df["Average Total Purchase Per Person"] = avg_purch_per_person_df
purchase_stats_df

# Reformat Columns
purchase_stats_df["Average Purchase Price"] = purchase_stats_df["Average Purchase Price"].map('${:.2f}'.format)
purchase_stats_df["Total_Purchase_value"] = purchase_stats_df["Total_Purchase_value"].map('${:.2f}'.format)
purchase_stats_df["Average Total Purchase Per Person"] = purchase_stats_df["Average Total Purchase Per Person"].map('${:.2f}'.format)
purchase_stats_df

,Purchase Counts,Average Purchase Price,Total_Purchase_value,Average Total Purchase Per Person
<10,23,$3.35,$77.13,$3.73
10-14,28,$2.96,$82.78,$3.17
15-19,136,$3.04,$412.89,$3.23
20-24,365,$3.05,$1114.06,$3.11
25-29,101,$2.90,$293.00,$2.77
30-34,73,$2.93,$214.00,$3.48
35-39,41,$3.60,$147.67,$4.07
40+,13,$2.94,$38.24,$6.76


In [53]:
#EIGHTH CELL
#VERIFEID AS WORKING VERIFIED FORMATTING
# TOP SPENDERS

# Create a summary data frame with SN and Count of purchases
top_df = raw_data_df.groupby("SN").count()
top_df

# Find mean purchase value
top_mean_df = raw_data_df.groupby("SN").mean()
top_mean_df

# Find total purchase value
top_total_df = raw_data_df.groupby("SN").sum()
top_total_df

# create new df = just columns of interest
#["Purchase Count"]
top_spenders_df = top_df[["Age"]]

# Rename Column Age as Purchase Count
top_spenders_df = top_spenders_df.rename(columns={"Age": "Purchase Count"})
top_spenders_df

# Combine Average Purchase Price and Total Purchase Value into top_spenders_df
top_spenders_df["Average Purchase Price"] = top_mean_df["Price"]
top_spenders_df["Total Purchase Value"] = top_total_df["Price"]
top_spenders_df

# Sort dataframe decending order
# DEBUG!! - sorting not working - removing axis=1 makes it work
top_spenders_df = top_spenders_df.sort_values(["Total Purchase Value"], ascending=False)

# Reformat numbers
top_spenders_df["Average Purchase Price"] = top_spenders_df["Average Purchase Price"].map('${:.2f}'.format)
top_spenders_df["Total Purchase Value"] = top_spenders_df["Total Purchase Value"].map('${:.2f}'.format)
top_spenders_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [14]:
#NINTH CELL
#MOST POPULAR ITEMS BY PURCHASE COUNT 
#VERIFIED AS WORKING AND VERIFIED FORMATTING

# Check if raw data has nan's - it does not
raw_data_df.describe()

# Find count of items purchased
popular_items_df = raw_data_df.groupby("Item ID").count()
popular_items_df["Purchase ID"]

# List unique Item Names
item_name_df = raw_data_df["Item Name"].unique()
item_name_df = pd.DataFrame(item_name_df)

# Get Average Item Price
item_price_df = raw_data_df.groupby("Item ID").mean()
item_price_df["Price"]

# Get Total Purchase Value
total_price_df = raw_data_df.groupby("Item ID").sum()
total_price_df["Price"]

# Get Item Name
item_name_df = raw_data_df.groupby("Item ID").first()
item_name_df["Item Name"]

# Combine results into a single dataframe
final_df = popular_items_df
final_df["Item Price"] = item_price_df["Price"]
final_df["Total Purchase Value"] = total_price_df["Price"]
final_df

# Remove Unneeded columns
del final_df["SN"]
del final_df["Age"]
del final_df["Gender"]
del final_df["Item Name"]
del final_df["Price"]

# Rename Purchase ID to Purchase Count
final_df = final_df.rename(columns={"Purchase ID": "Purchase Count"})

# Add Column Item Name
final_df["Item Name"] = item_name_df["Item Name"]

# Resort by Purchase ID
final_df = final_df.sort_values(["Purchase Count"], ascending=False)
final_df

# Reorder Columns
final_df = final_df[["Item Name", "Purchase Count", "Item Price", "Total Purchase Value"]]
final_df

# Reformat Item Price and Total Purchase Value
final_df["Item Price"] = final_df["Item Price"].map('${:.2f}'.format)
final_df["Total Purchase Value"] = final_df["Total Purchase Value"].map('${:.2f}'.format)
final_df.head()


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [93]:
# ALTERNATE WAY TO CREATE DATAFRAME - EASIER TO READ
pd.DataFrame({
    'Purchase Count': popular_items_df["Purchase ID"],
    'Item Price': item_price_df["Price"],
    'Total Purchase Value': total_price_df['Price']
})

,Purchase Count,Item Price,Total Purchase Value
Item ID,,,
0,4,1.2800,5.12
1,4,2.9425,11.77
2,6,2.4800,14.88
3,6,2.4900,14.94
4,5,1.7000,8.50
...,...,...,...
178,12,4.2300,50.76
179,6,4.4800,26.88
181,5,1.6600,8.30


In [15]:
#TENTH CELL
# MOST PROFITABLE ITEMS
# VERIFIED AS WORKING AND VERIFIED FORMATTING

# Resort Descending by Total Purchase Value
final_df = final_df.sort_values(["Total Purchase Value"], ascending=False)
final_df.head()


,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
63,Stormfury Mace,2,$4.99,$9.98
29,"Chaos, Ender of the End",5,$1.98,$9.90
173,Stormfury Longsword,2,$4.93,$9.86
38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48
143,Frenzied Scimitar,6,$1.56,$9.36
